In [ ]:
#libraries
from pandas_datareader import data as pdr
import pandas as pd
import numpy as np
import cvxpy as cvx
from scipy.stats import norm
from numpy.linalg import inv
import math

#for plotting
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import matplotlib.pyplot as plt

# Initialize plotly offline
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Data functions

In [ ]:
# Download Stock data from Yahoo Finance
def download_stock_data(start_date, end_date, asset_list, filename):
    for i, asset in enumerate(asset_list):
        if i == 0:
            data = pdr.get_data_yahoo(asset_list[i], start_date,end_date)[['Adj Close']]
            all_prices = data.rename(columns={"Adj Close": asset})
            
        else:
            new_asset = pdr.get_data_yahoo(asset_list[i], start_date,end_date)[['Adj Close']]
            all_prices = all_prices.join(new_asset.rename(columns={"Adj Close": asset}))
    
    all_prices.to_csv('./00_Data/' + filename + '.csv')
    return

# Extract Return and accumulated price data (starting value 1) from raw stock prices
def process_stock_data(all_prices):
    n_time, n_assets = all_prices.shape
    #Calculate returns
    all_returns = all_prices.pct_change(1).apply(lambda a : a + 1)
    
    # Calculate relative price time series
    all_prices_norm = all_returns.copy()
    for i in range(n_time):
        if i ==0:
            all_prices_norm.iloc[0] = np.ones(n_assets)
        else:
            for j in range(n_assets):
                if math.isnan(all_prices_norm.iloc[i,j]):
                    all_prices_norm.iloc[i,j] = all_prices_norm.iloc[i-1,j]
                else:
                    all_prices_norm.iloc[i,j] = all_prices_norm.iloc[i,j] * all_prices_norm.iloc[i-1,j]
                    
    return all_prices_norm, all_returns.dropna()

#1/fluc, 1, 1/fluc, 1, ...
def teststock1(T, fluc):
    stock = np.ones(T+1)
    for i in range(T+1):
        if i%2 == 1:
            stock[i] = 1.0/fluc
    return stock

#Brownian Motion
def teststock2(T, delta, sigma):
    stock1 = np.zeros(T+1)
    stock1[0] = 3.0
    for i in range(T):
        stock1[i+1] = stock1[i] + sigma + norm.rvs(scale=delta**2)
        if (stock1[i+1] <= 0):
            print('Negative Warning!')
    return stock1

#Garbage 1, 1/fluc, 1/pow(fluc,2) ,...
def teststock3(T, discount):
    stock = np.ones(T+1, dtype=float)
    for i in range(T):
        stock[i+1] = stock[i]*discount
    return stock

## Download and/or load Data

In [ ]:
# download stock data and safe as csv in /Data folder
commodities = ['GC=F','SI=F', 'CL=F'] #gold, silver, Crude Oil
bonds = ['^FVX'] #5 Year US Government Bonds
us_stocks = ['SPY', # The SP500 as a benchmark
             'AAPL', 'MMM', 'BA' , 'CAT', 'CVX' , 'CSCO', 'KO' , 'DIS',
             'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MSFT', 'PFE'] #Some stocks from the Dow Jones

start_data = '2000-01-01'
end_date = '2018-11-01'

#Download and safe data as csv
#download_stock_data(start_data, end_date, commodities, 'Commodities_latest')
#download_stock_data(start_data, end_date, bonds, 'bonds_latest')
#download_stock_data(start_data, end_date, us_stocks, 'us_stocks_latest')

In [ ]:
# # Read in the Historical Data
# GovBonds = pd.read_csv('Data/10YearUsGovBondYield.csv', header=2)[['Date','Close']]
# BruggeWheat = pd.read_csv('Data/BruggesWheatPrice.csv', header=2)[['Date','Close']]
# Gold = pd.read_csv('Data/Gold_Price.csv', header=2)[['Date','Close']]
# MSCIWorld = pd.read_csv('Data/MSCI_World.csv', header=2)[['Date','Close']]
# SPY = pd.read_csv('Data/S&P500.csv', header=2)[['Date','Close']]
# USCorpBonds = pd.read_csv('Data/UsCorporateBonds.csv', header=2)[['Date','Close']]
# UsReit = pd.read_csv('Data/UsREIT.csv', header=2)[['Date','Close']]
# UsWheat = pd.read_csv('Data/UsWheatPrice.csv', header=2)[['Date','Close']]

# # Modify datatype
# GovBonds['Date'] = pd.to_datetime(GovBonds['Date'], format="%m/%d/%Y")
# MSCIWorld['Date'] = pd.to_datetime(MSCIWorld['Date'], format="%m/%d/%Y")
# SPY['Date'] = pd.to_datetime(SPY['Date'], format="%m/%d/%Y")
# USCorpBonds['Date'] = pd.to_datetime(USCorpBonds['Date'], format="%m/%d/%Y")
# UsReit['Date'] = pd.to_datetime(UsReit['Date'], format="%m/%d/%Y")
# UsWheat['Date'] = pd.to_datetime(UsWheat['Date'], format="%m/%d/%Y")

In [ ]:
# read in the data again
us_stock_prices = pd.read_csv('./00_Data/us_stocks_latest.csv', header=0, index_col = 0)
us_stock_prices.index = pd.to_datetime(us_stock_prices.index)

#Process Stock Data
stock_prices_norm, stock_returns = process_stock_data(us_stock_prices)

# Extract benchmark SP500 Data
SPY_benchmark = stock_prices_norm.drop(us_stocks[1:], axis=1)
stock_prices_norm_wo_SPY = stock_prices_norm.drop(us_stocks[0], axis=1)

# filter out benchmark data from stock returns
stock_returns = stock_returns.drop(us_stocks[0], axis=1)

## Plot Data

In [ ]:
# Plotprices
iplot(us_stock_prices.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Prices',title='Prices'))

# Plot returns
iplot(stock_returns.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Returns',title='Returns'))

# Plot normalized prices
iplot(stock_prices_norm.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Relative Price',title='Relative Prices'))

# Algorithm Functions

## Online Optimization: First Order Method

In [ ]:
def First_Order_Method(stock_returns, D, G):
    asset_returns = stock_returns.as_matrix()
    T, m = asset_returns.shape
    T = T + 1
    
    p = np.zeros((T-1,m))
    p[0,:] = np.ones(m) / m
    reward = np.ones(T)
    reward[0] = np.dot(p[0], asset_returns[0])
    
    for i in range(T-2):
        eta = D/(G*np.sqrt(i+1))
        grad = -asset_returns[i] / np.dot(p[i], asset_returns[i])
        tmp1 = p[i] - eta*grad
        
        tmp2 = cvx.Variable(m)
        obj = cvx.Minimize(cvx.norm(tmp2-tmp1))
        constr = [tmp2 >= 0, np.ones(m)*tmp2 == 1]
        prob = cvx.Problem(obj, constr)
        prob.solve()
        p[i+1] = tmp2.value
    
    for i in range(T-1):
        reward[i+1] = reward[i] * np.dot(p[i], asset_returns[i])
    
    # create nice returns Weights Dataframe for plotting
    P = pd.DataFrame(data=p, index=stock_returns.index, columns=list(stock_returns))  
        
    return P, reward

## Online Optimization: First Order Method, allow short

In [ ]:
def First_Order_Method_Short(stock_returns, D, G):
    asset_returns = stock_returns.as_matrix()
    T, m = asset_returns.shape
    T = T + 1
    m = 2*m
    
    tmp = np.zeros((T-1,m))
    tmp[:,:int(m/2)] = asset_returns
    tmp[:,int(m/2):] = np.reciprocal(asset_returns)
    asset_returns = tmp
    
    p = np.zeros((T-1,m))
    #for i in range(int(m/2)):
    #    p[0,i] = 1/int(m/2)
    p[0,:] = np.ones(m) / m
    reward = np.ones(T)
    reward[0] = np.dot(p[0], asset_returns[0])
    
    for i in range(T-2):
        eta = D/(G*np.sqrt(i+1))
        grad = -asset_returns[i] / np.dot(p[i], asset_returns[i])
        tmp1 = p[i] - eta*grad
        
        tmp2 = cvx.Variable(m)
        obj = cvx.Minimize(cvx.norm(tmp2-tmp1))
        constr = [tmp2 >= 0, np.ones(m)*tmp2 == 1]
        prob = cvx.Problem(obj, constr)
        prob.solve()
        p[i+1] = tmp2.value
    
    for i in range(T-1):
        reward[i+1] = reward[i] * np.dot(p[i], asset_returns[i])
    
    # create nice returns Weights Dataframe for plotting
    P = pd.DataFrame(data=p, index=stock_returns.index, columns=['AAPL', 'MMM', 'BA' , 'CAT', 'CVX' , 'CSCO', 'KO' , 'DIS',
        'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MSFT', 'PFE', 'AAPL_short', 'MMM_short', 'BA_short' , 'CAT_short', 'CVX_short',
        'CSCO_short', 'KO_short' , 'DIS_short', 'GS_short', 'IBM_short', 'INTC_short', 'JNJ_short', 'JPM_short', 'MSFT_short', 'PFE_short'])  
        
    return P, reward

## Online Optimization: Second Order Method

In [ ]:
def Second_Order_Method(stock_returns, D, G, alpha=1.0):
    gamma = 0.5*min(alpha, 1.0/(4.0*D*G))
    eps = 1.0/pow(D*gamma,2)
    asset_returns = stock_returns.as_matrix()
    T, m = asset_returns.shape
    T = T + 1
    
    p = np.zeros((T-1,m))
    p[0,:] = np.ones(m) / m
    A = eps * np.eye(m)
    reward = np.ones(T)
    reward[0] = np.dot(p[0], asset_returns[0])
    
    for i in range(T-2):
        #print(i)
        grad = -asset_returns[i] / np.dot(p[i], asset_returns[i])
        #print(grad)
        #print(p[i])
        #print(np.dot(p[i], x[i]))
        A = A + np.dot(grad.reshape((m,1)), grad.reshape((1,m)))
        #print(A)
        tmp1 = p[i] - 1.0/gamma * np.dot(inv(A), grad)
        
        tmp2 = cvx.Variable(m)
        obj = cvx.Minimize(cvx.quad_form(tmp2-tmp1, A))
        constr = [tmp2 >= 0, np.ones(m)*tmp2 == 1]
        prob = cvx.Problem(obj, constr)
        prob.solve()
        #print(tmp2.value)
        p[i+1] = tmp2.value
    
    for i in range(T-1):
        reward[i+1] = reward[i] * np.dot(p[i], asset_returns[i])
    
    # create nice returns Weights Dataframe for plotting
    P = pd.DataFrame(data=p, index=stock_returns.index, columns=list(stock_returns))  
        
    return P, reward

## Online Optimization: Second Order Method, allow short

In [ ]:
def Second_Order_Method_Short(stock_returns, D, G, alpha=1.0):
    gamma = 0.5*min(alpha, 1.0/(4.0*D*G))
    eps = 1.0/pow(D*gamma,2)
    asset_returns = stock_returns.as_matrix()
    T, m = asset_returns.shape
    T = T + 1
    m = 2*m
    
    tmp = np.zeros((T-1,m))
    tmp[:,:int(m/2)] = asset_returns
    tmp[:,int(m/2):] = np.reciprocal(asset_returns)
    asset_returns = tmp
    
    p = np.zeros((T-1,m))
    #for i in range(int(m/2)):
    #    p[0,i] = 1/int(m/2)
    p[0,:] = np.ones(m) / m
    A = eps * np.eye(m)
    reward = np.ones(T)
    reward[0] = np.dot(p[0], asset_returns[0])
    
    for i in range(T-2):
        #print(i)
        grad = -asset_returns[i] / np.dot(p[i], asset_returns[i])
        #print(grad)
        #print(p[i])
        #print(np.dot(p[i], x[i]))
        A = A + np.dot(grad.reshape((m,1)), grad.reshape((1,m)))
        #print(A)
        tmp1 = p[i] - 1.0/gamma * np.dot(inv(A), grad)
        
        tmp2 = cvx.Variable(m)
        obj = cvx.Minimize(cvx.quad_form(tmp2-tmp1, A))
        constr = [tmp2 >= 0, np.ones(m)*tmp2 == 1]
        prob = cvx.Problem(obj, constr)
        prob.solve()
        #print(tmp2.value)
        p[i+1] = tmp2.value
    
    for i in range(T-1):
        reward[i+1] = reward[i] * np.dot(p[i], asset_returns[i])
    
    # create nice returns Weights Dataframe for plotting
    P = pd.DataFrame(data=p, index=stock_returns.index, columns=['AAPL', 'MMM', 'BA' , 'CAT', 'CVX' , 'CSCO', 'KO' , 'DIS',
        'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MSFT', 'PFE', 'AAPL_short', 'MMM_short', 'BA_short' , 'CAT_short', 'CVX_short',
        'CSCO_short', 'KO_short' , 'DIS_short', 'GS_short', 'IBM_short', 'INTC_short', 'JNJ_short', 'JPM_short', 'MSFT_short', 'PFE_short'])  
          
    return P, reward

In [ ]:
#Set D & G for SOM and FOM.
D = 2.0 #Bound the convex set.
G = 2.0 #Bound the gradient.

## Uniform Constant Rebalanced Portfolio

In [ ]:
def UniformConstantRebalancedPortfolio_general(stock_returns):
    asset_returns = stock_returns.as_matrix()
    T, m = asset_returns.shape
    T = T + 1

    #ratio for each asset
    r = 1.0/m

    # Set the uniform constant portfolio weighting
    W = np.repeat(r,m)
    
    # Calculate the wealth factors array over time
    wealth_factors = np.ones(T)
    for i in range(1,T):
        wealth_factors[i] = wealth_factors[i-1]*np.dot(W, asset_returns[i-1,:])
            
    return wealth_factors

## Best Constant Rebalanced Portfolio

In [ ]:
def BCRP_cvx(stock_returns):
    asset_returns = stock_returns.as_matrix()
    T, m = asset_returns.shape
    T = T + 1
    #print(T)

    w = cvx.Variable(m)
    S = 0
    for i in range(T-1):
        S += cvx.log(asset_returns[i,:]*w)
    objective = cvx.Maximize(S)
    constraints = [cvx.sum(w) == 1, w >= 0]

    prob = cvx.Problem(objective, constraints)
    prob.solve()  # Returns the optimal value.

    # get the optimal constant weight vector
    w_nom = w.value
    print(w_nom)
    
    #calculate the development of the CRP
    wealth_factors = np.ones(T)
    for i in range(1,T):
        wealth_factors[i] = wealth_factors[i-1]*np.dot(w_nom, asset_returns[i-1,:])
    
    #Weights = pd.DataFrame(data=w_nom, columns=list(stock_returns.columns.values))  
    #print(len(wealth_factors))
    return w_nom, wealth_factors

## EG Strategy based on MW

In [ ]:
def EG_Strategy_general(stock_returns, lower_bound = 0.90, upper_bound = 1.10):
    """
    Executes the EG Strategy backwards on data
    assets is an T row and m column Matrix where each row presents the value of an asset m at time T

    lower and upper bound are used to calculate a learning rate that gives guarantees on the performance
    """
    asset_returns = stock_returns.as_matrix()
    lower_bound = 0.90
    upper_bound = 1.10
    T, m = asset_returns.shape
    T = T + 1

    #Set learning rate
    # For the returns of each asset in each period roughly being bounded between lower and upper
    nu = (lower_bound/upper_bound)*math.sqrt(8*np.log(m)/T)
    #print(nu)

    # Set the Portfolio Weights
    W = np.zeros((T-1,m))
    #set initial weights
    r = 1.0/m
    W[0,:] = np.repeat(r,m)

    wealth_factors = np.ones(T)
    # Update wealth and weight for the next round
    for i in range(T-1):
        if i == 0:
            #calculate wealth payoff for first round
            wealth_factors[1] = np.dot(W[0,:], asset_returns[0,:])
        else:
            # update weights
            for j in range(m):
                W[i,j] = W[i-1,j]*math.exp(nu*asset_returns[i-1,j]/(np.dot(W[i-1,:],asset_returns[i-1,:])))
            norm_weight = np.sum(W[i,:], axis=0)
            #normalize weights so that they sum to one
            for j in range(m):
                W[i,j] = W[i,j]/norm_weight
            # calculate accumulated wealth payoff for the rounds
            wealth_factors[i+1] = wealth_factors[i]*np.dot(W[i,:], asset_returns[i,:])
    
    # create nice returns Weights Dataframe for plotting
    Weights = pd.DataFrame(data=W, index=stock_returns.index, columns=list(stock_returns))  
    
    return Weights, wealth_factors

## EG Strategy based on MW, allow short

In [ ]:
def EG_Strategy_general_Short(stock_returns, lower_bound = 0.90, upper_bound = 1.10):
    """
    Executes the EG Strategy backwards on data
    assets is an T row and m column Matrix where each row presents the value of an asset m at time T

    lower and upper bound are used to calculate a learning rate that gives guarantees on the performance
    """
    asset_returns = stock_returns.as_matrix()
    lower_bound = 0.90
    upper_bound = 1.10
    T, m = asset_returns.shape
    T = T + 1
    m = 2*m
    
    tmp = np.zeros((T-1,m))
    tmp[:,:int(m/2)] = asset_returns
    tmp[:,int(m/2):] = np.reciprocal(asset_returns)
    asset_returns = tmp

    #Set learning rate
    # For the returns of each asset in each period roughly being bounded between lower and upper
    nu = (lower_bound/upper_bound)*math.sqrt(8*np.log(m)/T)
    #print(nu)

    # Set the Portfolio Weights
    W = np.zeros((T-1,m))
    #set initial weights
    r = 1.0/m
    W[0,:] = np.repeat(r,m)

    wealth_factors = np.ones(T)
    # Update wealth and weight for the next round
    for i in range(T-1):
        if i == 0:
            #calculate wealth payoff for first round
            wealth_factors[1] = np.dot(W[0,:], asset_returns[0,:])
        else:
            # update weights
            for j in range(m):
                W[i,j] = W[i-1,j]*math.exp(nu*asset_returns[i-1,j]/(np.dot(W[i-1,:],asset_returns[i-1,:])))
            norm_weight = np.sum(W[i,:], axis=0)
            #normalize weights so that they sum to one
            for j in range(m):
                W[i,j] = W[i,j]/norm_weight
            # calculate accumulated wealth payoff for the rounds
            wealth_factors[i+1] = wealth_factors[i]*np.dot(W[i,:], asset_returns[i,:])
    
    # create nice returns Weights Dataframe for plotting
    Weights = pd.DataFrame(data=W, index=stock_returns.index, columns=['AAPL', 'MMM', 'BA' , 'CAT', 'CVX' , 'CSCO', 'KO' , 'DIS',
        'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MSFT', 'PFE', 'AAPL_short', 'MMM_short', 'BA_short' , 'CAT_short', 'CVX_short',
        'CSCO_short', 'KO_short' , 'DIS_short', 'GS_short', 'IBM_short', 'INTC_short', 'JNJ_short', 'JPM_short', 'MSFT_short', 'PFE_short'])  
    
    return Weights, wealth_factors

## EG Strategy based on MW, AdaHedge

In [ ]:
def EG_Strategy_general_AdaHedge(stock_returns, lower_bound = 0.90, upper_bound = 1.10):
    """
    Executes the EG Strategy backwards on data
    assets is an T row and m column Matrix where each row presents the value of an asset m at time T

    lower and upper bound are used to calculate a learning rate that gives guarantees on the performance
    """
    asset_returns = stock_returns.as_matrix()
    lower_bound = 0.90
    upper_bound = 1.10
    T, m = asset_returns.shape
    T = T + 1
    
    # Mixibility Gap
    Delta = 5.0
    #Delta = 0.0
    deltat = np.zeros(T-1)
    # learning rate
    nut = np.zeros(T-1)
    nut[0] = 20.0
    # l introduced in the book
    l = np.zeros(m)
    
    # Set the Portfolio Weights
    W = np.zeros((T-1,m))
    # set initial weights
    r = 1.0/m
    W[0,:] = np.repeat(r,m)

    wealth_factors = np.ones(T)
    # Update wealth and weight for the next round
    for i in range(T-1):
        if i == 0:
            #calculate wealth payoff for first round
            wealth_factors[1] = np.dot(W[0,:], asset_returns[0,:])
            for j in range(m):
                l[j] = upper_bound/lower_bound - asset_returns[0,j]/np.dot(W[0,:], asset_returns[0,:])
            deltat[0] = np.dot(l,W[0,:]) + (np.log(np.dot(W[0,:], np.exp(-nut[0]*l))))/nut[0]
            Delta += deltat[0]
        else:
            # Update learning rate
            nut[i] = np.log(m)/Delta
            # update weights
            for j in range(m):
                W[i,j] = W[i-1,j]*math.exp(nut[i]*asset_returns[i-1,j]/(np.dot(W[i-1,:],asset_returns[i-1,:])))
            norm_weight = np.sum(W[i,:], axis=0)
            #normalize weights so that they sum to one
            for j in range(m):
                W[i,j] = W[i,j]/norm_weight
            # calculate accumulated wealth payoff for the rounds
            wealth_factors[i+1] = wealth_factors[i]*np.dot(W[i,:], asset_returns[i,:])
            for j in range(m):
                l[j] = upper_bound/lower_bound - asset_returns[i,j]/np.dot(W[i,:], asset_returns[i,:])
            deltat[i] = np.dot(l,W[i,:]) + (np.log(np.dot(W[i,:], np.exp(-nut[i]*l))))/nut[i]
            Delta += deltat[i]
        #print(Delta)
        #print(deltat[i])
        #print(l)
        #print(nut[i])
            
    # create nice returns Weights Dataframe for plotting
    Weights = pd.DataFrame(data=W, index=stock_returns.index, columns=list(stock_returns))  
    
    return Weights, wealth_factors

## EG Strategy based on MW, AdaHedge, allow short

In [ ]:
def EG_Strategy_general_AdaHedge_Short(stock_returns, lower_bound = 0.90, upper_bound = 1.10):
    """
    Executes the EG Strategy backwards on data
    assets is an T row and m column Matrix where each row presents the value of an asset m at time T

    lower and upper bound are used to calculate a learning rate that gives guarantees on the performance
    """
    asset_returns = stock_returns.as_matrix()
    lower_bound = 0.90
    upper_bound = 1.10
    T, m = asset_returns.shape
    T = T + 1
    m = 2*m
    
    tmp = np.zeros((T-1,m))
    tmp[:,:int(m/2)] = asset_returns
    tmp[:,int(m/2):] = np.reciprocal(asset_returns)
    asset_returns = tmp
    
    # Mixibility Gap
    Delta = 5.0
    #Delta = 0.0
    deltat = np.zeros(T-1)
    # learning rate
    nut = np.zeros(T-1)
    nut[0] = 20.0
    # l introduced in the book
    l = np.zeros(m)
    
    # Set the Portfolio Weights
    W = np.zeros((T-1,m))
    #set initial weights
    r = 1.0/m
    W[0,:] = np.repeat(r,m)

    wealth_factors = np.ones(T)
    # Update wealth and weight for the next round
    for i in range(T-1):
        if i == 0:
            #calculate wealth payoff for first round
            wealth_factors[1] = np.dot(W[0,:], asset_returns[0,:])
            for j in range(m):
                l[j] = upper_bound/lower_bound - asset_returns[0,j]/np.dot(W[0,:], asset_returns[0,:])
            deltat[0] = np.dot(l,W[0,:]) + (np.log(np.dot(W[0,:], np.exp(-nut[0]*l))))/nut[0]
            Delta += deltat[0]
        else:
            # Update learning rate
            nut[i] = np.log(m)/Delta
            # update weights
            for j in range(m):
                W[i,j] = W[i-1,j]*math.exp(nut[i]*asset_returns[i-1,j]/(np.dot(W[i-1,:],asset_returns[i-1,:])))
            norm_weight = np.sum(W[i,:], axis=0)
            #normalize weights so that they sum to one
            for j in range(m):
                W[i,j] = W[i,j]/norm_weight
            # calculate accumulated wealth payoff for the rounds
            wealth_factors[i+1] = wealth_factors[i]*np.dot(W[i,:], asset_returns[i,:])
            for j in range(m):
                l[j] = upper_bound/lower_bound - asset_returns[i,j]/np.dot(W[i,:], asset_returns[i,:])
            deltat[i] = np.dot(l,W[i,:]) + (np.log(np.dot(W[i,:], np.exp(-nut[i]*l))))/nut[i]
            Delta += deltat[i]
        #print(Delta)
        #print(deltat[i])
        #print(l)
        #print(nut[i])
            
    # create nice returns Weights Dataframe for plotting
    Weights = pd.DataFrame(data=W, index=stock_returns.index, columns=['AAPL', 'MMM', 'BA' , 'CAT', 'CVX' , 'CSCO', 'KO' , 'DIS',
        'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MSFT', 'PFE', 'AAPL_short', 'MMM_short', 'BA_short' , 'CAT_short', 'CVX_short',
        'CSCO_short', 'KO_short' , 'DIS_short', 'GS_short', 'IBM_short', 'INTC_short', 'JNJ_short', 'JPM_short', 'MSFT_short', 'PFE_short'])  
    
    return Weights, wealth_factors

## Universal Portfolio (not efficient)

In [ ]:
def Universal_Portfolio(stock1, stock2, N, T):
    h = 1.0/N
    weight = np.ones(N+1) * 2
    weight[0] = 1
    weight[1] = 1
    #t = 0, 1, ..., T
    x = np.zeros((T,2))
    for i in range(T):
        x[i,0] = stock1[i+1]/stock1[i]
        x[i,1] = stock2[i+1]/stock2[i]
    balance = np.ones((N+1,T))
    for i in range(N+1):
        balance[i,0] = np.dot([h*i, 1.0-h*i], x[0])
    for i in range(N+1):
        for j in range(T-1):
            balance[i,j+1] = np.dot([h*i, 1.0-h*i], x[j+1]) * balance[i][j]
    
    unip = np.zeros((T,2))
    unip[0] = [0.5,0.5]
    for i in range(T-1):
        tmp1 = np.dot(balance[:,i], weight) / (2.0*N)
        tmp2 = np.dot(np.multiply(np.reshape(balance[:,i],(1,N+1)), np.arange(0,1+h/2.0,h)), weight) / (2.0*N)
        unip[i+1,0] = tmp2/tmp1
        unip[i+1,1] = 1-unip[i+1,0]
    
    unip_reward = np.ones(T)
    unip_reward[0] = np.dot(unip[0], x[0])
    for i in range(T-1):
        unip_reward[i+1] = unip_reward[i] * np.dot(unip[i+1], x[i+1])
    #print(unip_reward[T-1])
    #print(np.dot(balance[:,T-1], weight) / (2.0*N))
    
    balance_reward = np.amax(balance, axis=0)
    
    regret = np.zeros(T)
    for i in range(T):
        regret[i] = (np.log(balance_reward[i]) - np.log(unip_reward[i])) / (i+1)
        
    return unip, unip_reward, balance_reward, regret

## Toy Data

In [ ]:
#Toy
'''
T = 1000
delta = 0.1
sigma1 = 0.001
sigma2 = -0.001
#s1 = teststock1(T, 2.0)
s1 = teststock3(T, 0.99)
s2 = teststock1(T, 1.0)
#s1 = teststock2(T, delta, sigma1)
#s2 = teststock2(T, delta, sigma2)
#s3 = teststock2(T,delta)
#cash = np.ones(T+1)
assets = np.array([s1, s2])
#assets = np.array([s1, s2, s3, cash])
'''

In [ ]:
#Toy
'''
#Plot the market
Ts = np.arange(T+1)
plt.figure(figsize=(20,10))
plt.plot(Ts, s1, label="stock1")
plt.plot(Ts, s2, label="stock2")
#plt.plot(Ts, s3, label="stock3")
#plt.ylim(0,1000)
plt.xlabel("Time")
plt.ylabel("Price")
plt.grid()
plt.title("Brownian Motion")
#plt.title("Toy")
plt.legend(loc=2)
plt.show()
'''

In [ ]:
'''
D = 2.0
G = 2.0
SOM_p, SOM_reward = Second_Order_Method(assets, D, G)
FOM_p, FOM_reward = First_Order_Method(assets, D, G)
EG_p, EG_reward = EG_Strategy_general(assets)
UCRP_reward = UniformConstantRebalancedPortfolio_general(assets)
Uni_p, Uni_reward, balance_reward, Uni_regret = Universal_Portfolio(s1, s2, 40, T)
#BCRP_p, BCRP_reward = BCRP_cvx(asset)
#print(EG_p)
'''

In [ ]:
'''
Ts = np.arange(T)
plt.figure(figsize=(20,10))
plt.plot(Ts, FOM_reward, label="First_Order_Method")
plt.plot(Ts, SOM_reward, label="Second_Order_Method")
plt.plot(Ts, EG_reward, label="EG_Strategy")
plt.plot(Ts, UCRP_reward, label="UCRP")
plt.plot(Ts, balance_reward, label="BCRP")
plt.plot(Ts, Uni_reward, label="Universal")
#plt.ylim(0,1000)
plt.xlabel("Time")
plt.ylabel("Reward")
plt.grid()
plt.title("Brownian Motion")
#plt.title("Fluctuation")
plt.legend(loc=2)
plt.show()
'''

## Tests on real market data

In [ ]:
# Solves the convex optimization problem of the Best constant rebalanced portfolio in hindsight
BCRP, SPY_benchmark['BCRP_reward'] = BCRP_cvx(stock_returns)

In [ ]:
# Uniform Constant Rebalanced Portfolio
SPY_benchmark['UCRP_reward'] = UniformConstantRebalancedPortfolio_general(stock_returns)

In [ ]:
# EG Strategy
EG_Weights, SPY_benchmark['EG_reward'] = EG_Strategy_general(stock_returns)
EG_Ada_Weights, SPY_benchmark['EG_Ada_reward']= EG_Strategy_general_AdaHedge(stock_returns)

In [ ]:
# First and Second order method
FOM_Weights, SPY_benchmark['FOM_reward'] = First_Order_Method(stock_returns, D, G)
SOM_Weights, SPY_benchmark['SOM_reward'] = Second_Order_Method(stock_returns, D, G)

In [ ]:
# Consider short
EG_Short_Weights, SPY_benchmark['EG_Short_reward'] = EG_Strategy_general_Short(stock_returns)
EG_Ada_Short_Weights, SPY_benchmark['EG_Ada_Short_reward']= EG_Strategy_general_AdaHedge_Short(stock_returns)
FOM_Short_Weights, SPY_benchmark['FOM_Short_reward'] = First_Order_Method_Short(stock_returns, D, G)
SOM_Short_Weights, SPY_benchmark['SOM_Short_reward'] = Second_Order_Method_Short(stock_returns, D, G)

In [ ]:
# Plot Portfolio algorithm performances and SPY Benchmark
iplot(SPY_benchmark.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Wealth Factor',title='Relative Wealth over time'))

In [ ]:
#Allow short
# Plot Weights for EG over time
iplot(EG_Short_Weights.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Weights',title='EG Weights over time, allow short'))

# Plot Weights for EG_Ada over time
iplot(EG_Ada_Short_Weights.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Weights',title='EG_Ada Weights over time, allow short'))

# Plot Weights for FOM over time
iplot(FOM_Short_Weights.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Weights',title='FOM Weights over time, allow short'))

# Plot Weights for SOM over time
iplot(SOM_Short_Weights.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Weights',title='SOM Weights over time, allow short'))

In [ ]:
# Plot Weights for EG over time
iplot(EG_Weights.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Weights',title='EG Weights over time'))

# Plot Weights for EG_Ada over time
iplot(EG_Ada_Weights.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Weights',title='EG_Ada Weights over time'))

# Plot Weights for FOM over time
iplot(FOM_Weights.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Weights',title='FOM Weights over time'))

# Plot Weights for SOM over time
iplot(SOM_Weights.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Weights',title='SOM Weights over time'))